# **Análise de Sentimentos de _Reviews_ de produtos na plataforma Steam**

## **Importando bibliotecas**

In [1]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np 
import re
import string
import unidecode

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/juan-
[nltk_data]     burtet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Carregando _dataset_** 

In [3]:
data = pd.read_csv('dataset/steam_reviews.tsv', sep='\t')[['review', 'voted_up']]

In [4]:
data.describe()

,review,voted_up
count,507051,507052
unique,506693,2
top,Err:507,True
freq,226,460579


In [5]:
data.dtypes

review      object
voted_up      bool
dtype: object

In [6]:
data.head(5)

,review,voted_up
0,"Corrupted é um jogo de RPG bem simplificado, o...",True
1,Meu PC tem uma placa de vídeo equivalente a GT...,False
2,Muito Bom! O meu diz que é recomendado specs:...,True
3,Funciona,True
4,"Muito bom o Steam VR, Very good.",True


### **Separando Exemplos**

#### Selecionando apenas Exemplos Positivos

In [7]:
positive = data[data['voted_up'] == True].sample(46000)
positive

,review,voted_up
339826,"Socorro, minha vida nunca mais será a mesma, s...",True
1379,"me surpreendeu muito, é bem gostosa a historia...",True
466034,Recomendo muito para quem gosta de jogos pixel...,True
310599,O que dizer desse jogo louco onde se fica cham...,True
271821,Oq eu posso disser....ESSE JOGO ÉMARAVILHOSO É...,True
...,...,...
15318,Produto de alta qualidade que aceita mudanças ...,True
450607,Um bom jogo recomendo !,True
268175,Este é um excelente Point and Click pra se jog...,True
87257,Um jogo épico que envolve sobrevivência a um a...,True


#### Selecionando apenas Exemplos Negativos

In [8]:
negative = data[data['voted_up'] == False].sample(46000)
negative

,review,voted_up
428738,meu jogo não abre mais! ele parou de funcionar...,False
294866,Jogo cansado! Mais cansado que o malaquias fal...,False
51631,Chaaaaaaaaaaaaaaaaaaaaato.,False
323774,mds nao tem nem ligua portuguesa mds q jogo ruim,False
125443,não esta dando,False
...,...,...
38220,Ruim demais! Achei que minha placa de vídeo qu...,False
276199,Jogo toda hora atualizando e fazendo patchs qu...,False
424545,"Jogo bem fraco. Sinceramente, não acrescenta e...",False
411980,Não comprem.... Peca em coisas simples como co...,False


#### Concatenando os dados e misturando-os

In [9]:
frames = [positive, negative]
data = pd.concat(frames)

In [10]:
data = data.sample(frac=1).reset_index(drop=True)

In [11]:
data

,review,voted_up
0,Não funcionou no meu PC e olha que tenho um i3...,False
1,Este jogo é para o publico que queira jogar co...,True
2,I regret having bought this game. It gave me a...,False
3,óooootimo compensa muito o valor pago !!!! nao...,True
4,"Jogo simples, gráficos honestos de simplicidad...",True
...,...,...
91995,"Muito ruim, péssimo, horrível, dinheiro jogado...",False
91996,"Eu gostei muito deste mod de Portal 2, talvez ...",True
91997,TOPTERMI,True
91998,"ótima DLC,conteúdo realmente novo(parece um no...",True


## **Processando os Dados**

In [12]:
def process_data(df):
    
    # To lowercase
    df['review'] = df['review'].str.lower()
    
    # Removing punctuations
    df['review'] = df['review'].str.replace('[^\w\s]','')
    
    # Remove a single character from the string
    remove_single_char = lambda x: re.sub(r"\b[a-zA-Z]\b", "", x)
    df['review'] = df['review'].apply(remove_single_char)
    
    # Substituting multiple spaces with single space
    remove_multiple_spaces = lambda x: ' '.join(x.split())
    df['review'] = df['review'].apply(remove_multiple_spaces)
    
    # Remove repeated letters if there 3 or more
    remove_repeated_letters = lambda x: re.sub(r'([a-z])\1+', r'\1\1', x)
    df['review'] = df['review'].apply(remove_repeated_letters)
    
    # Removing accents
    remove_accent = lambda x: unidecode.unidecode(x)
    df['review'] = df['review'].apply(remove_accent)
    
    return df

In [13]:
data = process_data(data)
data

,review,voted_up
0,nao funcionou no meu pc olha que tenho um i3 e...,False
1,este jogo e para publico que queira jogar com ...,True
2,regret having bought this game it gave me head...,False
3,oootimo compensa muito valor pago nao enjoa ne...,True
4,jogo simples graficos honestos de simplicidade...,True
...,...,...
91995,muito ruim pessimo horrivel dinheiro jogado fora,False
91996,eu gostei muito deste mod de portal 2 talvez s...,True
91997,toptermi,True
91998,otima dlcconteudo realmente novoparece um novo...,True


# TF-IDF

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = set(nltk.corpus.stopwords.words('portuguese'))


corpus = data['review'].values
# vectorizer = TfidfVectorizer(stop_words=stop_words)
# X = vectorizer.fit_transform(corpus)

# print(vectorizer.get_feature_names())
# print(X.toarray()) 

['00', '000', '0000', '000000000', '00000000000000', '0000000000000000', '0000000000000000000', '0000000000000000000000000000000000000000000000000000000000000000000000010', '0000000000000000001', '0000000000004e93', '0000000000052f86', '00000000000oo', '00000001', '0000001', '000000se', '000001', '000007', '00001', '000010', '0001', '000100007e71', '0001000080d1', '00010000832c', '000100008529', '000100009efd', '0001000115af', '000100034429', '000100034d40', '000100038e8c', '00010003f083', '00010003f746', '00010003f7d6', '00010003f7ff', '000100064478', '000100076ec0', '0001000b1c03', '0001001653e6', '0001001d399c', '0001001d744d', '0001001dc25d', '0001001dc30c', '0001001dc42d', '0001001dc69f', '0001001f5f6a', '0001001f71ff', '0001001f7e90', '000222c2', '0005da12', '000710', '0007810000', '000xx0', '001', '0010', '00100000', '00101100', '00113', '0015', '0017h', '0018e2bc', '0018ed04', '0018ed10', '0018eda4', '0018ee48', '0018ee78', '0018ef40', '0018f42c', '0018f444', '0018f71c', '0018f

MemoryError: Unable to allocate array with shape (92000, 104105) and data type float64